In [22]:
import os
import sys


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Darts
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, smape
from darts.models import TFTModel
from darts.utils.likelihood_models import QuantileRegression
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.timeseries_generation import holidays_timeseries

# ------------------------------------------------------------
# 0) 변수 설정
# ------------------------------------------------------------

# 경로 
train_path = "DATA/train.csv"
test_path = "DATA/test.csv"

# 건물번호
building_num = 1

# 타겟 컬럼
target_col = "전력소비량(kWh)"
col_names = ["building_num", "datetime", "temp", "rain", "wind", "humidity", "sun_hr", "sun_mj", "power"]

# 데이터 빈도
FREQ = "h"
INPUT_CHUNK = 168        # 과거 7일 창
OUTPUT_CHUNK = 24        # 24시간 예측
USE_GPU = True

# 모델 파라미터
QUANTILES = [0.1, 0.5, 0.9]
N_EPOCHS = 50
BATCH_SIZE = 64


In [27]:
# ------------------------------------------------------------
# 1) 기본 유틸리티 함수
# ------------------------------------------------------------

def load_data(file_path: str):
    df = pd.read_csv(file_path, header=0, index_col=0)
    return df


def split_by_building(df: pd.DataFrame, building_num: int):
    if "건물번호" not in df.columns:
        return df[df['building_num'] == building_num]
    return df[df['건물번호'] == building_num]


def change_col_name(df: pd.DataFrame, col_names: list[str]=None):
    if col_names is None: # 일조, 일사 추가
        col_names = ["building_num", "datetime", "temp", "rain", "wind", "humidity", "sun_hr", "sun_mj", "power"]
    df.columns = col_names
    return df


def plot_data(df: pd.DataFrame, target_col: str, building_num: int):
    plt.rcParams['font.family'] = 'NanumGothic'
    plt.figure(figsize=(15, 5))
    df[target_col].plot(title=f"건물번호 {building_num} - {target_col}")
    plt.show()


# ------------------------------------------------------------
# 2) Darts 유틸리티 함수
# ------------------------------------------------------------

def to_timeseries(df: pd.DataFrame, target_cols: list[str], time_col: str, freq: str = "h") -> TimeSeries:
    """
    df를 TimeSeries로 변환
    """
    ts = TimeSeries.from_dataframe(
        df.sort_values(time_col), 
        time_col=time_col,
        value_cols=target_cols,
        freq=freq
    )
    return ts

# Test Code
df = load_data("DATA/train.csv")
df_cols = df.columns.tolist(); print(df_cols)
target_cols = col_names.remove("datetime");print(target_cols)
df_01 = split_by_building(df, 1)
df_01 = change_col_name(df_01); print(df_01.columns)

# print(df_01.head())
ts = to_timeseries(
    df = df_01, 
    target_cols = col_names,
    time_col = "datetime", 
    freq = "h"
)
ts

['건물번호', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)']
None
Index(['building_num', 'datetime', 'temp', 'rain', 'wind', 'humidity',
       'sun_hr', 'sun_mj', 'power'],
      dtype='object')


<TimeSeries (datetime: 2040, component: 7, sample: 1)> Size: 114kB
array([[[1.83000e+01],
        [0.00000e+00],
        [2.60000e+00],
        ...,
        [0.00000e+00],
        [0.00000e+00],
        [5.79480e+03]],

       [[1.83000e+01],
        [0.00000e+00],
        [2.70000e+00],
        ...,
        [0.00000e+00],
        [0.00000e+00],
        [5.59185e+03]],

       [[1.81000e+01],
        [0.00000e+00],
        [2.60000e+00],
        ...,
...
        ...,
        [0.00000e+00],
        [0.00000e+00],
        [4.13586e+03]],

       [[2.73000e+01],
        [0.00000e+00],
        [1.00000e+00],
        ...,
        [0.00000e+00],
        [0.00000e+00],
        [4.57197e+03]],

       [[2.68000e+01],
        [0.00000e+00],
        [4.00000e-01],
        ...,
        [0.00000e+00],
        [0.00000e+00],
        [4.49901e+03]]])
Coordinates:
  * datetime   (datetime) datetime64[ns] 16kB 2024-06-01 ... 2024-08-24T23:00:00
  * component  (component) object 56B 'temp' 'rain' 'wind' ... 'sun_mj' 'power'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None

In [ ]:


def preprocess_data(df: pd.DataFrame):
    pass


def pipeline_data(df: pd.DataFrame):
    pipeline = Pipeline(
        


def main():
    print("Hello from data-with-darts!")
    df = load_data("DATA/train.csv")
    df_01 = split_by_building(df, 1)
    print(df_01.head())
    plot_data(df_01, "전력소비량(kWh)", 1)

    preprocess_data(df_01)

main()

SyntaxError: '(' was never closed (2825136078.py, line 6)